In [ ]:
import cv2
import numpy as np
import time

# Load YOLOv3-tiny model and its configuration
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

# Load class names
classes = []
with open("coco.txt", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Get the output layer names from the YOLO model
layer_names = net.getLayerNames()

# Depending on the OpenCV version, net.getUnconnectedOutLayers() might return a 1D array or a 2D array
try:
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
except IndexError:
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Generate random colors for each class
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Initialize video capture from the default camera
cap = cv2.VideoCapture(0)

# Set font for text on the output video
font = cv2.FONT_HERSHEY_SIMPLEX

# Track the starting time and frame ID
starting_time = time.time()
frame_id = 0

while True:
    # Read frames from the video capture
    _, frame = cap.read()
    frame_id += 1
    height, width, channels = frame.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Initialize lists for detected objects' details
    class_ids = []
    confidences = []
    boxes = []

    # Process the outputs from the network
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.2:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Perform non-maximum suppression to eliminate redundant overlapping boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.8, 0.3)
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 30), font, 1, color, 3)

    # Calculate and display frames per second (FPS)
    elapsed_time = time.time() - starting_time
    fps = frame_id / elapsed_time
    cv2.putText(frame, "FPS: " + str(round(fps, 2)), (10, 50), font, 1, (0, 255, 0), 3)

    # Display the output frame
    cv2.imshow("Object Detection", frame)

    # Exit if the 'Esc' key is pressed
    key = cv2.waitKey(1)
    if key == 27:
        break

# Release the capture and destroy all windows
cap.release()
cv2.destroyAllWindows()